In [2]:
import pandas as pd

def extract_tables_from_excel(file_path):
    """
    Extracts tables from an Excel file by identifying groups of contiguous non-empty rows.
    Handles edge cases like empty rows, multi-row headers, and partial tables.
    """
    # Load the Excel file
    workbook = pd.ExcelFile(file_path)

    # Initialize a dictionary to store tables from all sheets
    all_tables = {}

    # Iterate over each sheet
    for sheet_name in workbook.sheet_names:
        sheet_data = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
        
        # Variables to track tables
        tables = []
        current_table = []
        current_table_start_row = None  # Track where the table starts for metadata
        
        # Iterate through rows to detect tables based on blank rows
        for index, row in sheet_data.iterrows():
            if row.isnull().all():
                if current_table:
                    # Finalize current table
                    table_df = pd.DataFrame(current_table).reset_index(drop=True)
                    tables.append({"data": table_df, "start_row": current_table_start_row})
                    current_table = []
                    current_table_start_row = None
            else:
                if current_table_start_row is None:
                    current_table_start_row = index  # Set the starting row for the new table
                current_table.append(row)
        
        # Add the last table if any
        if current_table:
            table_df = pd.DataFrame(current_table).reset_index(drop=True)
            tables.append({"data": table_df, "start_row": current_table_start_row})
        
        # Store tables from this sheet
        all_tables[sheet_name] = tables

    return all_tables

# Example usage
file_path = '/Users/ajay/Documents/Atomic/inventory_analysis/Data/Company 1 - Inventory Planning.xlsx'
all_tables = extract_tables_from_excel(file_path)

# Display extracted tables or save them
for sheet, tables in all_tables.items():
    print(f"Sheet: {sheet}")
    for i, table_info in enumerate(tables):
        print(f"\nTable {i+1} from {sheet} (Starting row: {table_info['start_row']})")
        print(table_info["data"].head())


Sheet: Detail2-The Mozz Junior-Lineage

Table 1 from Detail2-The Mozz Junior-Lineage (Starting row: 0)
                    0                       1                  2   \
0          PickUp Date                Location               Type   
1  2023-08-13 00:00:00  Lineage - Columbus, OH            True Up   
2  2023-08-17 00:00:00  Lineage - Columbus, OH            True Up   
3  2023-08-23 00:00:00  Lineage - Columbus, OH  Customer Delivery   
4  2023-09-01 00:00:00  Lineage - Columbus, OH           Transfer   

            3          4         5        6                    7       8   \
0  Destination  Transport     BOL #     PO #  Product Description  Cases    
1          NaN        NaN       NaN      NaN      The Mozz Junior    2660   
2          NaN        NaN       NaN      NaN      The Mozz Junior      70   
3  Ben E Keith        HMB  20000221   646204      The Mozz Junior   -1050   
4          NaN   From RSF  20000209  3000002      The Mozz Junior     700   

        9         1

In [3]:
all_tables.keys()

dict_keys(['Detail2-The Mozz Junior-Lineage', 'TK Copy of Inventory Summary - ', 'INVENTORY SNAPSHOT', 'Sheet24', '1016 OWNEDCOMARCO SNAP', 'Current Inventory', 'INPUT -> DOT Inventory Report', 'Inventory Detail', 'True Up Template', 'Copy of Inventory Detail', 'Production by Month', 'Delivery by Month', 'Delivery by Customer', 'Inventory Detail 2024', 'Production and Delivery', 'RSF Check', 'Inventory - Monthly NEW', 'TK True Up Summary', 'Instructions', 'Item List', 'Invoice Tracker(TK)', 'Locations'])

In [4]:

def extract_tables_with_metadata(file_path):
    """
    Extracts tables from an Excel file, assigns table IDs, and captures metadata like
    sheet name, columns, formulas, and dependencies.
    """
    # Load the Excel file
    workbook = pd.ExcelFile(file_path)

    # Initialize a dictionary to store all extracted tables and metadata
    all_tables = {}
    table_metadata = {}

    # Table ID counter
    table_id = 1

    # Iterate over each sheet
    for sheet_name in workbook.sheet_names:
        sheet_data = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
        
        # Variables to track tables
        tables = []
        current_table = []
        current_table_start_row = None  # Track where the table starts for metadata
        
        # Iterate through rows to detect tables based on blank rows
        for index, row in sheet_data.iterrows():
            if row.isnull().all():
                if current_table:
                    # Finalize current table
                    table_df = pd.DataFrame(current_table).reset_index(drop=True)
                    tables.append((table_df, current_table_start_row))
                    current_table = []
                    current_table_start_row = None
            else:
                if current_table_start_row is None:
                    current_table_start_row = index  # Set the starting row for the new table
                current_table.append(row)
        
        # Add the last table if any
        if current_table:
            table_df = pd.DataFrame(current_table).reset_index(drop=True)
            tables.append((table_df, current_table_start_row))
        
        # Process extracted tables for metadata
        for table_df, start_row in tables:
            # Assign a unique table ID
            current_table_id = table_id
            table_id += 1

            # Extract column names from the first row
            if not table_df.empty:
                columns = list(table_df.iloc[0])
            else:
                columns = []

            # Analyze formulas and dependencies
            column_metadata = {}
            dependencies = set()

            for col_index, column_name in enumerate(columns):
                column_metadata[column_name] = {"Formula": None, "Dependencies": []}

                for row_index, value in table_df.iloc[1:].iterrows():
                    if isinstance(value[col_index], str) and value[col_index].startswith("="):
                        column_metadata[column_name]["Formula"] = value[col_index]
                        # Example of extracting dependencies (simplified)
                        if "!" in value[col_index]:
                            sheet_dependency = value[col_index].split("!")[0].strip("=")
                            dependencies.add(sheet_dependency)
            
            # Store metadata for the table
            table_metadata[current_table_id] = {
                "SheetName": sheet_name,
                "Columns": columns,
                "ColumnMetadata": column_metadata,
                "Dependencies": list(dependencies),
            }

            # Add the table data to all_tables
            all_tables[current_table_id] = table_df.reset_index(drop=True)

    return all_tables, table_metadata

# Example usage
file_path = '/Users/ajay/Documents/Atomic/inventory_analysis/Data/Company 1 - Inventory Planning.xlsx'
extracted_tables, table_metadata = extract_tables_with_metadata(file_path)


In [5]:
for 

{1: {'SheetName': 'Detail2-The Mozz Junior-Lineage',
  'Columns': ['PickUp Date',
   'Location',
   'Type',
   'Destination',
   'Transport',
   'BOL #',
   'PO #',
   'Product Description',
   'Cases ',
   'Pallets',
   'FTL',
   'Product UPC',
   'Product Code'],
  'ColumnMetadata': {'PickUp Date': {'Formula': None, 'Dependencies': []},
   'Location': {'Formula': None, 'Dependencies': []},
   'Type': {'Formula': None, 'Dependencies': []},
   'Destination': {'Formula': None, 'Dependencies': []},
   'Transport': {'Formula': None, 'Dependencies': []},
   'BOL #': {'Formula': None, 'Dependencies': []},
   'PO #': {'Formula': None, 'Dependencies': []},
   'Product Description': {'Formula': None, 'Dependencies': []},
   'Cases ': {'Formula': None, 'Dependencies': []},
   'Pallets': {'Formula': None, 'Dependencies': []},
   'FTL': {'Formula': None, 'Dependencies': []},
   'Product UPC': {'Formula': None, 'Dependencies': []},
   'Product Code': {'Formula': None, 'Dependencies': []}},
  'Depe

## Better

In [7]:
import openpyxl
import pandas as pd
from openpyxl.utils import get_column_letter

def extract_tables_with_metadata_openpyxl(file_path):
    """
    Extracts tables from an Excel file using OpenPyXL, capturing metadata like:
    - Sheet name
    - Columns
    - Cell formulas
    - Dependencies on other sheets
    """
    # Load the workbook with formulas
    workbook = openpyxl.load_workbook(file_path, data_only=False,read_only=True)

    # Initialize result dictionaries
    table_metadata = {}
    table_id = 1  # Unique ID for each table

    # Iterate through each sheet in the workbook
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]

        # Track the current table
        current_table = []
        current_table_start_row = None
        columns = []
        dependencies = set()

        # Iterate over rows to identify tables and formulas
        for row_index, row in enumerate(sheet.iter_rows(values_only=False), start=1):
            # Check if the row is empty
            if all(cell.value is None for cell in row):
                if current_table:  # End of table
                    # Store metadata for the current table
                    table_metadata[table_id] = {
                        "SheetName": sheet_name,
                        "Columns": columns,
                        "Dependencies": list(dependencies),
                    }
                    table_id += 1
                    current_table = []
                    columns = []
                    dependencies = set()
                    current_table_start_row = None
            else:
                if not current_table_start_row:  # Start of a new table
                    current_table_start_row = row_index

                # Add row data
                current_table.append([cell.value for cell in row])

                # Extract columns from the first row of the table
                if not columns and row_index == current_table_start_row:
                    columns = [cell.value for cell in row]

                # Analyze formulas and dependencies
                for cell in row:
                    if cell.data_type == 'f':  # Check for formulas
                        if cell.value and "!" in cell.value:
                            # Extract dependency sheet
                            dep_sheet = cell.value.split("!")[0].strip("=")
                            dependencies.add(dep_sheet)

        # Handle the last table if not empty
        if current_table:
            table_metadata[table_id] = {
                "SheetName": sheet_name,
                "Columns": columns,
                "Dependencies": list(dependencies),
            }
            table_id += 1

    return table_metadata

# Example usage
file_path = '/Users/ajay/Documents/Atomic/inventory_analysis/Data/Company 1 - Inventory Planning.xlsx'
metadata = extract_tables_with_metadata_openpyxl(file_path)



In [8]:
metadata

{1: {'SheetName': 'Detail2-The Mozz Junior-Lineage',
  'Columns': ['PickUp Date',
   'Location',
   'Type',
   'Destination',
   'Transport',
   'BOL #',
   'PO #',
   'Product Description',
   'Cases ',
   'Pallets',
   'FTL',
   'Product UPC',
   'Product Code'],
  'Dependencies': []},
 2: {'SheetName': 'TK Copy of Inventory Summary - ',
  'Columns': [None,
   None,
   'INSERT DATE OF TRUE UP BELOW',
   None,
   None,
   None,
   None,
   None,
   None,
   None],
  'Dependencies': ["vlookup(A3,'Item List'", "sumifs('Inventory Detail'"]},
 3: {'SheetName': 'TK Copy of Inventory Summary - ',
  'Columns': ['The Mozz Junior',
   "=vlookup(A7,'Item List'!A:C,2,false)",
   '=sumifs(\'Inventory Detail\'!$K:$K,\'Inventory Detail\'!$J:$J,$A7,\'Inventory Detail\'!$C:$C,"<="&C$2)',
   '=sumifs(\'Inventory Detail\'!$K:$K,\'Inventory Detail\'!$J:$J,$A7,\'Inventory Detail\'!$C:$C,"<="&D$2)',
   '=sumifs(\'Inventory Detail\'!$K:$K,\'Inventory Detail\'!$J:$J,$A7,\'Inventory Detail\'!$C:$C,"<="&E$2)'

## Best till now

In [16]:
import openpyxl
import pandas as pd

def extract_tables_with_metadata_openpyxl(file_path):
    """
    Extracts tables from an Excel file using OpenPyXL in read-only mode, capturing metadata like:
    - Sheet name
    - Columns
    - Cell formulas and dependencies
    """
    # Load the workbook with formulas in read-only mode
    workbook = openpyxl.load_workbook(file_path, data_only=False, read_only=True)

    # Initialize result dictionaries
    table_metadata = {}
    table_id = 1  # Unique ID for each table

    # Iterate through each sheet in the workbook
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]

        # Track the current table
        current_table = []
        current_table_start_row = None
        columns = []
        sheet_dependencies = set()
        column_metadata = {}

        # Iterate over rows to identify tables and formulas
        for row_index, row in enumerate(sheet.iter_rows(), start=1):
            # Check if the row is empty
            if all(cell.value is None for cell in row):
                if current_table:  # End of table
                    # Store metadata for the current table
                    table_metadata[table_id] = {
                        "SheetName": sheet_name,
                        "Columns": columns,
                        "Dependencies": list(sheet_dependencies),
                        "ColumnMetadata": column_metadata,
                    }
                    table_id += 1
                    current_table = []
                    columns = []
                    sheet_dependencies = set()
                    column_metadata = {}
                    current_table_start_row = None
            else:
                if not current_table_start_row:  # Start of a new table
                    current_table_start_row = row_index

                # Add row data
                current_table.append([cell.value for cell in row])

                # Extract columns from the first row of the table
                if not columns and row_index == current_table_start_row:
                    columns = [cell.value for cell in row]
                    # Initialize column metadata structure
                    for col in columns:
                        if col:
                            column_metadata[col] = {"Formulas": None, "Dependencies": []}

                # Analyze formulas and dependencies
                for col_index, cell in enumerate(row):
                    if cell.data_type == 'f':  # Check for formulas
                        formula = cell.value
                        column_name = columns[col_index] if col_index < len(columns) else None
                        if column_name:
                            if column_metadata[column_name]["Formulas"] is None:
                                column_metadata[column_name]["Formulas"] = formula
                            if "!" in formula:  # Extract inter-sheet dependencies
                                dep_sheet = formula.split("!")[0].strip("=")
                                sheet_dependencies.add(dep_sheet)
                                column_metadata[column_name]["Dependencies"].append(dep_sheet)

        # Handle the last table if not empty
        if current_table:
            table_metadata[table_id] = {
                "SheetName": sheet_name,
                "Columns": columns,
                "Dependencies": list(sheet_dependencies),
                "ColumnMetadata": column_metadata,
            }
            table_id += 1

    return table_metadata

# Example usage
file_path = '/Users/ajay/Documents/Atomic/inventory_analysis/Data/Company 1 - Inventory Planning.xlsx'
metadata = extract_tables_with_metadata_openpyxl(file_path)

# Display metadata


In [17]:
metadata

{1: {'SheetName': 'Detail2-The Mozz Junior-Lineage',
  'Columns': ['PickUp Date',
   'Location',
   'Type',
   'Destination',
   'Transport',
   'BOL #',
   'PO #',
   'Product Description',
   'Cases ',
   'Pallets',
   'FTL',
   'Product UPC',
   'Product Code'],
  'Dependencies': [],
  'ColumnMetadata': {'PickUp Date': {'Formulas': None, 'Dependencies': []},
   'Location': {'Formulas': None, 'Dependencies': []},
   'Type': {'Formulas': None, 'Dependencies': []},
   'Destination': {'Formulas': None, 'Dependencies': []},
   'Transport': {'Formulas': None, 'Dependencies': []},
   'BOL #': {'Formulas': None, 'Dependencies': []},
   'PO #': {'Formulas': None, 'Dependencies': []},
   'Product Description': {'Formulas': None, 'Dependencies': []},
   'Cases ': {'Formulas': None, 'Dependencies': []},
   'Pallets': {'Formulas': None, 'Dependencies': []},
   'FTL': {'Formulas': None, 'Dependencies': []},
   'Product UPC': {'Formulas': None, 'Dependencies': []},
   'Product Code': {'Formulas': 

## Cleaner than prev


In [13]:
import openpyxl
import difflib  # For similarity comparison


def is_formula_different(existing_formulas, new_formula, threshold=0.7):
    """
    Checks if a new formula is significantly different from existing ones.
    Uses a similarity threshold.
    """
    for formula in existing_formulas:
        similarity = difflib.SequenceMatcher(None, formula, new_formula).ratio()
        if similarity >= threshold:  # Similar enough, treat as the same
            return False
    return True


def extract_tables_with_metadata_openpyxl(file_path):
    """
    Extracts tables from an Excel file using OpenPyXL in read-only mode, capturing metadata like:
    - Sheet name
    - Columns
    - Unique formulas per column (if significantly different)
    - Dependencies on other sheets
    """
    # Load the workbook with formulas in read-only mode
    workbook = openpyxl.load_workbook(file_path, data_only=False, read_only=True)

    # Initialize result dictionaries
    table_metadata = {}
    table_id = 1  # Unique ID for each table

    # Iterate through each sheet in the workbook
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]

        # Track the current table
        current_table = []
        current_table_start_row = None
        columns = []
        sheet_dependencies = set()
        column_metadata = {}

        # Iterate over rows to identify tables and formulas
        for row_index, row in enumerate(sheet.iter_rows(), start=1):
            # Check if the row is empty
            if all(cell.value is None for cell in row):
                if current_table:  # End of table
                    # Store metadata for the current table
                    table_metadata[table_id] = {
                        "SheetName": sheet_name,
                        "Columns": columns,
                        "Dependencies": list(sheet_dependencies),
                        "ColumnMetadata": column_metadata,
                    }
                    table_id += 1
                    current_table = []
                    columns = []
                    sheet_dependencies = set()
                    column_metadata = {}
                    current_table_start_row = None
            else:
                if not current_table_start_row:  # Start of a new table
                    current_table_start_row = row_index

                # Add row data
                current_table.append([cell.value for cell in row])

                # Extract columns from the first row of the table
                if not columns and row_index == current_table_start_row:
                    columns = [cell.value for cell in row]
                    # Initialize column metadata structure
                    for col in columns:
                        if col:
                            column_metadata[col] = {"Formulas": [], "Dependencies": []}

                # Analyze formulas and dependencies
                for col_index, cell in enumerate(row):
                    if cell.data_type == 'f':  # Check for formulas
                        formula = cell.value
                        column_name = columns[col_index] if col_index < len(columns) else None
                        if column_name:
                            # Add the formula if it is too different
                            if is_formula_different(column_metadata[column_name]["Formulas"], formula):
                                column_metadata[column_name]["Formulas"].append(formula)
                                if "!" in formula:  # Extract inter-sheet dependencies
                                    dep_sheet = formula.split("!")[0].strip("=")
                                    sheet_dependencies.add(dep_sheet)
                                    column_metadata[column_name]["Dependencies"].append(dep_sheet)

        # Handle the last table if not empty
        if current_table:
            table_metadata[table_id] = {
                "SheetName": sheet_name,
                "Columns": columns,
                "Dependencies": list(sheet_dependencies),
                "ColumnMetadata": column_metadata,
            }
            table_id += 1

    return table_metadata


# Example usage
file_path = '/Users/ajay/Documents/Atomic/inventory_analysis/Data/Company 1 - Inventory Planning.xlsx'
metadata = extract_tables_with_metadata_openpyxl(file_path)


In [14]:
metadata

{1: {'SheetName': 'Detail2-The Mozz Junior-Lineage',
  'Columns': ['PickUp Date',
   'Location',
   'Type',
   'Destination',
   'Transport',
   'BOL #',
   'PO #',
   'Product Description',
   'Cases ',
   'Pallets',
   'FTL',
   'Product UPC',
   'Product Code'],
  'Dependencies': [],
  'ColumnMetadata': {'PickUp Date': {'Formulas': [], 'Dependencies': []},
   'Location': {'Formulas': [], 'Dependencies': []},
   'Type': {'Formulas': [], 'Dependencies': []},
   'Destination': {'Formulas': [], 'Dependencies': []},
   'Transport': {'Formulas': [], 'Dependencies': []},
   'BOL #': {'Formulas': [], 'Dependencies': []},
   'PO #': {'Formulas': [], 'Dependencies': []},
   'Product Description': {'Formulas': [], 'Dependencies': []},
   'Cases ': {'Formulas': [], 'Dependencies': []},
   'Pallets': {'Formulas': [], 'Dependencies': []},
   'FTL': {'Formulas': [], 'Dependencies': []},
   'Product UPC': {'Formulas': [], 'Dependencies': []},
   'Product Code': {'Formulas': [], 'Dependencies': []}}

In [15]:
import openpyxl
from collections import defaultdict

def extract_excel_tables(file_path):
    """
    Extracts tables, formulas and dependencies from Excel file.
    Uses read-only mode for better performance.
    """
    # Load workbook in read-only mode
    wb = openpyxl.load_workbook(file_path, read_only=True, data_only=False)
    tables = defaultdict(dict)
    table_count = 0

    # Process each sheet
    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]
        current_table = []
        formulas = defaultdict(list)
        dependencies = set()
        
        # Track table boundaries
        in_table = False
        headers = None

        # Process rows
        for row in sheet.rows:
            row_data = [cell.value for cell in row]
            
            # Check if row is empty
            if all(val is None for val in row_data):
                if in_table:
                    # Save current table and reset
                    if current_table and headers:
                        table_count += 1
                        tables[table_count] = {
                            'sheet': sheet_name,
                            'headers': headers,
                            'data': current_table[1:],  # Exclude header row
                            'formulas': dict(formulas),
                            'dependencies': list(dependencies)
                        }
                    current_table = []
                    formulas.clear()
                    dependencies.clear()
                    headers = None
                    in_table = False
                continue

            # Start new table
            if not in_table:
                in_table = True
                headers = [str(val) if val is not None else f'Col{i}' 
                          for i, val in enumerate(row_data)]

            # Process formulas and dependencies
            for i, cell in enumerate(row):
                if cell.data_type == 'f':
                    formula = str(cell.value)
                    col_name = headers[i] if headers else f'Col{i}'
                    
                    # Only add unique formulas
                    if formula not in formulas[col_name]:
                        formulas[col_name].append(formula)
                    
                    # Check for sheet references
                    if '!' in formula:
                        ref_sheet = formula.split('!')[0].strip("'")
                        dependencies.add(ref_sheet)

            current_table.append(row_data)

        # Handle last table in sheet
        if current_table and headers:
            table_count += 1
            tables[table_count] = {
                'sheet': sheet_name,
                'headers': headers,
                'data': current_table[1:],
                'formulas': dict(formulas),
                'dependencies': list(dependencies)
            }

    wb.close()
    return dict(tables)

def print_table_info(tables):
    """Pretty print the extracted table information."""
    for table_id, table in tables.items():
        print(f"\n=== Table {table_id} ===")
        print(f"Sheet: {table['sheet']}")
        print(f"Headers: {', '.join(table['headers'])}")
        
        if table['formulas']:
            print("\nFormulas:")
            for col, formulas in table['formulas'].items():
                print(f"\n{col}:")
                for formula in formulas:
                    print(f"  {formula}")
        
        if table['dependencies']:
            print(f"\nDepends on sheets: {', '.join(table['dependencies'])}")
        
        print(f"Data rows: {len(table['data'])}")
        print("-" * 50)

if __name__ == "__main__":
    # file_path = "Company 1  Inventory Planning.xlsx"
    try:
        tables = extract_excel_tables(file_path)
        print_table_info(tables)
    except Exception as e:
        print(f"Error: {str(e)}")

Error: list index out of range
